In [44]:
import random as rd
import math
import numpy as np
import numpy.random as nprd
%matplotlib inline
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
import scipy
pd.options.mode.chained_assignment = None 

In [71]:
def arrivals(events, rate, numb_samples):
    y = nprd.gamma(events,rate,numb_samples)
    return y

def items(trials, prob_of_success, numb_samples):
    y = nprd.negative_binomial(trials, prob_of_success, numb_samples)
    return y

def processing_time(constant_waiting, item_waiting, list_items):
    y = list(item_waiting*np.array(list_items) + constant_waiting)
    return y

class checkout_line:
    """
    An object which accounts for a line up to the checkout
    
    Uses negative binomial distribution for number of items, with inital item purchase goal (r), and 
    probability of buying other items (p)
    
    Assumes a poisson process with average rate of arrival (lamb)
    
    cw    = Real number representing minimum wait time for processing 
    pw    = Real number representing per item waiting time for processing
    piw   = cw + pw*p*r/(1-p) (average waiting time for processing)
    items = average number of items purchased
    
    """
    
    def __init__(self, const_wait_time, per_item_wait_time, rate_arrival, \
                 prob_of_other_buys, initial_item):
        
        self.p = 1 - prob_of_other_buys
        self.r = initial_item
        self.items = self.r*self.p/(1-self.p) + self.r
        self.cw = const_wait_time
        self.pw = per_item_wait_time
        self.piw = self.cw + self.pw*self.items
        
        if 1/rate_arrival > self.cw + self.items*self.piw:
            print('Warning: stability is not possible')
        
        self.lamb = rate_arrival
        
        
        

    
    def create_timeseires(self, numb_samples):
        
        """
        Takes random variable paramters in checkout_line object and number of samples wanted (numb_samples) and outputs
        coresponding dataframe, with columns associted with; time of arrival, number of items, processing time, time 
        of exit, waiting time, cashier (final three columns filled with NaN)
        
        n = number of samples to create in timeseries
        
        
        """
        
        self.n = numb_samples
        #Datarame for all arrials
        matrix = pd.DataFrame(np.zeros((self.n, 6)), columns=["time_arrive", "num_items", \
                                                              "processing_time","time_left", "waiting_time", "cashier"])
        
        #Setting up number of itmes for all samples
        matrix["num_items"] = items(self.r,self.p, self.n) + self.r
        
        #Setting up processing time for all samples
        matrix["processing_time"] = processing_time(self.cw, self.pw, matrix["num_items"])

        #Setting up arrival time for all samples
        matrix["time_arrive"] = arrivals(1,1/self.lamb, self.n)
        matrix["time_arrive"] = matrix["time_arrive"].cumsum()
        
        self.df = matrix
    
    def checkout_1(checkout_line_object):
        """
        Checkout system with one line, 5 cashiers, and first person in line goes to cashier with none at it

        """
        #start off with all checkouts being empty
        cashiers_open = [0,0,0,0,0]
        mat = checkout_line_object.df

        for i in range(0,checkout_line_object.n):
            #if there are empty cashiers
            if (mat[i:i+1:]["time_arrive"]).iloc[0] > min(cashiers_open):

                #time left = timea arrival + prcoessing time = next time this cashier will be open
                a = mat[i:i+1:]["time_arrive"].iloc[0] + mat[i:i+1:]["processing_time"].iloc[0]
                mat[i:i+1:]["time_left"].iloc[0] = a
                min_waiting_cashier = cashiers_open.index(min(cashiers_open)) #index of open cashier
                mat[i:i+1:]["cashier"].iloc[0] = min_waiting_cashier + 1 #recording which cashier used
                cashiers_open[min_waiting_cashier] = a

                #waiting time is the same as processing time
                mat[i:i+1:]["waiting_time"].iloc[0] = mat[i:i+1:]["processing_time"].iloc[0]

            #if there are no empty cashier
            else:
                #time left is time earliest cashier to open + processing time
                a =  min(cashiers_open) + mat[i:i+1:]["processing_time"].iloc[0]
                mat[i:i+1:]["time_left"].iloc[0] = a
                min_waiting_cashier = cashiers_open.index(min(cashiers_open)) #index of earliest open cashier
                mat[i:i+1:]["cashier"].iloc[0] = min_waiting_cashier + 1 #recording which cashier used
                cashiers_open[min_waiting_cashier] = a

                #waiting time is the difference between time fo arrival and time left
                mat[i:i+1:]["waiting_time"].iloc[0] = a - mat[i:i+1:]["time_arrive"].iloc[0]
        return mat
    
    def checkout_2(checkout_line_object):
        """
        Checkout system with 5 line, 5 cashiers, and person arrives goes to cashier with min number people in line

        """
        #start off with all checkouts being empty
        cashiers_open = [0,0,0,0,0]
        cashier_times = [[0],[0],[0],[0],[0]]
        mat = checkout_line_object.df

        for i in range(0,checkout_line_object.n):
            #looking at who is in line
            temp = mat[:i:]
            arrive_time = mat[i:i+1:]["time_arrive"].iloc[0]
            current_line = temp[temp["time_left"] > arrive_time]
            waiting_customers = current_line["cashier"]

            #count how many people are in each line
            cashiers_open = [sum(waiting_customers == 1),sum(waiting_customers == 2),\
            sum(waiting_customers == 3), sum(waiting_customers == 4), sum(waiting_customers == 5)]

            #set cashier based on cashier with min people in line
            min_waiting_cashier = cashiers_open.index(min(cashiers_open)) #index of min people in line
            cashier = min_waiting_cashier + 1 #recording which cashier used
            mat[i:i+1:]["cashier"].iloc[0] =  cashier


            #if there are empty cashiers
            if 0 == min(cashiers_open):
                #time left = timea arrival + prcoessing time = next time this cashier will be open
                a = arrive_time + mat[i:i+1:]["processing_time"].iloc[0]
                mat[i:i+1:]["time_left"].iloc[0] = a

                #waiting time is the same as processing time
                mat[i:i+1:]["waiting_time"].iloc[0] = mat[i:i+1:]["processing_time"].iloc[0]

            #if there are no empty cashier 
            else:
                #time left = time person in front at cashier left + processing time
                in_front_left = max(current_line[current_line["cashier"] == cashier]["time_left"])
                a =  in_front_left + mat[i:i+1:]["processing_time"].iloc[0]
                mat[i:i+1:]["time_left"].iloc[0] = a

                #waiting time is the difference between time fo arrival and time left
                mat[i:i+1:]["waiting_time"].iloc[0] = a - mat[i:i+1:]["time_arrive"].iloc[0]
        return mat
    
    def checkout_3(checkout_line_object):
        """
        Checkout system with 5 line, 5 cashiers, and  each customer randomly selects a line upon arrival at the checkout.

        """
        #start off with all checkouts being empty
        cashiers_open = [0,0,0,0,0]
        cashier_times = [[0],[0],[0],[0],[0]]
        mat = checkout_line_object.df

        for i in range(0,checkout_line_object.n):
            #looking at who is in line
            temp = mat[:i:]
            arrive_time = mat[i:i+1:]["time_arrive"].iloc[0]
            current_line = temp[temp["time_left"] > arrive_time]
            waiting_customers = current_line["cashier"]


            #set cashier based on sampling from descrete uniform distribution between 1 and 5 (closed)
            cashier = np.random.randint(1,6,size=1)[0] #index of cashier chosen and recording it
            mat[i:i+1:]["cashier"].iloc[0] =  cashier

            #if cashier is empty
            if 0 == sum(waiting_customers == cashier):
                #time left = timea arrival + prcoessing time = next time this cashier will be open
                a = arrive_time + mat[i:i+1:]["processing_time"].iloc[0]
                mat[i:i+1:]["time_left"].iloc[0] = a

                #waiting time is the same as processing time
                mat[i:i+1:]["waiting_time"].iloc[0] = mat[i:i+1:]["processing_time"].iloc[0]

            #if cashier is not empty
            else:
                #time left = time person in front at cashier left + processing time
                in_front_left = max(current_line[current_line["cashier"] == cashier]["time_left"])
                a =  in_front_left + mat[i:i+1:]["processing_time"].iloc[0]
                mat[i:i+1:]["time_left"].iloc[0] = a

                #waiting time is the difference between time fo arrival and time left
                mat[i:i+1:]["waiting_time"].iloc[0] = a - mat[i:i+1:]["time_arrive"].iloc[0]
        return mat
    
    def checkout_4(checkout_line_object, exp_items):
        """
        Checkout system with 4 standard lines, 1 express line (10 prodcuts or less). 

        Upon arrival each customer with more then exp_items items go to the line with least people and 
        to the express line otherwise .

        """
        #start off with all checkouts being empty
        cashiers_open = [0,0,0,0,0]
        cashier_times = [[0],[0],[0],[0],[0]]
        mat = checkout_line_object.df

        for i in range(0,checkout_line_object.n):
            #looking at who is in line
            temp = mat[:i:]
            arrive_time = mat[i:i+1:]["time_arrive"].iloc[0]
            current_line = temp[temp["time_left"] > arrive_time]
            waiting_customers = current_line["cashier"]

            #count how many customers are in each line and which line has least customers
            cashiers_open = [sum(waiting_customers == 1),sum(waiting_customers == 2),\
            sum(waiting_customers == 3), sum(waiting_customers == 4), sum(waiting_customers == 5)]


            #checking whether will use expres line otherwise will use line with min customers waiting
            cashier = 1
            num_items = mat[i:i+1:]["num_items"].iloc[0]
            if num_items > exp_items:
                #set cashier based on cashier with min people in line that is not an express cashier
                non_express_lines = cashiers_open[1:]
                min_waiting_cashier = non_express_lines.index(min(non_express_lines)) + 1 #index of wanted cashier
                cashier = min_waiting_cashier + 1 #recording which cashier used

            #set cashier that will be used
            mat[i:i+1:]["cashier"].iloc[0] =  cashier

            #if cashier is empty
            if 0 == sum(waiting_customers == cashier):
                #time left = timea arrival + prcoessing time = next time this cashier will be open
                a = arrive_time + mat[i:i+1:]["processing_time"].iloc[0]
                mat[i:i+1:]["time_left"].iloc[0] = a

                #waiting time is the same as processing time
                mat[i:i+1:]["waiting_time"].iloc[0] = mat[i:i+1:]["processing_time"].iloc[0]

            #if there are cashier is not empty
            else:
                #time left = time person in front at cashier left + processing time
                in_front_left = max(current_line[current_line["cashier"] == cashier]["time_left"])
                a =  in_front_left + mat[i:i+1:]["processing_time"].iloc[0]
                mat[i:i+1:]["time_left"].iloc[0] = a

                #waiting time is the difference between time fo arrival and time left
                mat[i:i+1:]["waiting_time"].iloc[0] = a - mat[i:i+1:]["time_arrive"].iloc[0]
        return mat
    
    def checkout_5(checkout_line_object, exp_items):
        """
        Checkout system with 4 standard lines, 1 express line (10 prodcuts or less). 

        Upon arrival each customer with more then exp_items items go to the line with least people or 
        to the express line if customer has less then exp_items and the express line has the least custmers in line .

        """
        #start off with all checkouts being empty
        cashiers_open = [0,0,0,0,0]
        cashier_times = [[0],[0],[0],[0],[0]]
        mat = checkout_line_object.df

        for i in range(0,checkout_line_object.n):

            #looking at who is in line
            temp = mat[:i:]
            arrive_time = mat[i:i+1:]["time_arrive"].iloc[0]
            current_line = temp[temp["time_left"] > arrive_time]
            waiting_customers = current_line["cashier"]

            #count how many customers are in each line and which line has least customers
            cashiers_open = [sum(waiting_customers == 1),sum(waiting_customers == 2),\
            sum(waiting_customers == 3), sum(waiting_customers == 4), sum(waiting_customers == 5)]
            min_waiting_cashier = cashiers_open.index(min(cashiers_open)) #index of min people in line

            #checking whether will use expres line or line with min customers waiting
            cashier = 1
            num_items = mat[i:i+1:]["num_items"].iloc[0]
            if num_items > exp_items or \
            (num_items <= exp_items and cashiers_open[0] > cashiers_open[min_waiting_cashier]):
                #set cashier based on cashier with min people in line that is not an express cashier
                non_express_lines = cashiers_open[1:]
                min_waiting_cashier = non_express_lines.index(min(non_express_lines)) + 1 #index of wanted cashier
                cashier = min_waiting_cashier + 1 #recording which cashier used

            #set cashier that will be used
            mat[i:i+1:]["cashier"].iloc[0] =  cashier


            #if cashier is empty
            if 0 == sum(waiting_customers == cashier):
                #time left = timea arrival + prcoessing time = next time this cashier will be open
                a = arrive_time + mat[i:i+1:]["processing_time"].iloc[0]
                mat[i:i+1:]["time_left"].iloc[0] = a

                #waiting time is the same as processing time
                mat[i:i+1:]["waiting_time"].iloc[0] = mat[i:i+1:]["processing_time"].iloc[0]

            #if there are cashier is not empty
            else:
                #time left = time person in front at cashier left + processing time
                in_front_left = max(current_line[current_line["cashier"] == cashier]["time_left"])
                a =  in_front_left + mat[i:i+1:]["processing_time"].iloc[0]
                mat[i:i+1:]["time_left"].iloc[0] = a

                #waiting time is the difference between time fo arrival and time left
                mat[i:i+1:]["waiting_time"].iloc[0] = a - mat[i:i+1:]["time_arrive"].iloc[0]
        return mat

In [ ]:
c1 = checkout_line(const_wait_time=1, per_item_wait_time=0.03, initial_item = 5, prob_of_other_buys= 0.6, 
              rate_arrival = 4 )

c1.create_timeseires(100)

In [70]:
checkout_1(c1)

,time_arrive,num_items,processing_time,time_left,waiting_time,cashier
0,0.057910,11,1.33,1.387910,1.330000,1.0
1,0.718782,20,1.60,2.318782,1.600000,2.0
2,1.201305,8,1.24,2.441305,1.240000,3.0
3,1.298391,13,1.39,2.688391,1.390000,4.0
4,1.323465,14,1.42,2.743465,1.420000,5.0
5,1.685807,9,1.27,2.955807,1.270000,1.0
6,2.135643,11,1.33,3.648782,1.513139,2.0
7,2.144872,5,1.15,3.591305,1.446433,3.0
8,2.478555,10,1.30,3.988391,1.509836,4.0
9,2.787400,10,1.30,4.087400,1.300000,5.0


# Unrelated but possibly helpful code

meanitems = 24 # the average number of items per customer
nbr = 4 # the r parameter for the negative binomial distribution
nbp = nbr / (meanitems-1+nbr) #prob


def G_light_N_S(light_time):
    t = light_time
    v_num = np.random.poisson(1.2*t)
    arive_t = sorted(np.random.uniform(0, t, v_num))
    car_pas = int(np.random.uniform(t,2*t))
    return (v_num, arive_t, car_pas)

def R_light_N_S():
    t = 30
    v_num = np.random.poisson(1.2*t)
    arive_t = sorted(np.random.uniform(0, t, v_num))
    return (v_num, arive_t)

def G_light_E_W():
    t = 30
    v_num = np.random.poisson(0.7*t)
    arive_t = sorted(np.random.uniform(0, t, v_num))
    car_pas = int(np.random.uniform(t,2*t))
    return v_num, arive_t, car_pas

def R_light_E_W(light_time):
    t = light_time
    v_num = np.random.poisson(0.7*t)
    arive_t = sorted(np.random.uniform(0, t, v_num))
    return (v_num, arive_t)

def calc_avg_wt(base_wc, base_wt, wc_new, wc_g):
    old = base_wc*base_wt
    new =  np.sum(wc_new[:wc_g])
    wt = (old + new)/(base_wc + wc_g) 
    return(wt)
    

def simulate(waiting_time):
    wt = waiting_time
    #start with empty interesection and unbiased waiting time ( waiting time 0)
    vect_car_EW = []
    vect_car_NS = []
    EW_wt = 0
    base_wc_EW = 0
    NS_wt = 0
    base_wc_NS = 0
    for i in range(0,500):
        #begin with G_EW, R_NS each period
        Vec_EW = G_light_E_W()
        Vec_NS = R_light_N_S()
        #number of cars arrived now EW
        new_w_c = Vec_EW[0]
        #num of cars going EW
        new_g = Vec_EW[2]
        #recalculating waiting time EW
        EW_wt = calc_avg_wt(base_wc_EW, EW_wt, vect_car_EW, new_g)
        base_wc_EW += new_g
        #number of cars waiting from before EW
        tick = len(vect_car_EW)
        #new waiting times for cars waiting EW
        vect_car_EW = [x + 30 for x in vect_car_EW[new_g:]] + [30 - x for x in Vec_EW[1][max(new_g - tick,0):]]
        #new waiting times for cars waiting NS
        vect_car_NS = [x + 30 for x in vect_car_NS] + [30 - x for x in Vec_NS[1]] 
        
                
        #then G_NS, R_EW 
        Vec_EW = R_light_E_W(wt)
        Vec_NS = G_light_N_S(wt)
        #number of cars arrived now NS
        new_w_c = Vec_NS[0]
        #num of cars going NS
        new_g = Vec_NS[2]
        #recalculating waiting time NS
        NS_wt = calc_avg_wt(base_wc_NS, NS_wt, vect_car_NS, new_g)
        base_wc_NS += new_g
        #number of cars waiting from before NS
        tick = len(vect_car_NS)
        #new waiting times for cars waiting NS
        vect_car_NS = [x + wt for x in vect_car_NS[new_g:]] + [wt - x for x in Vec_NS[1][max(new_g - tick,0):]]
        #new waiting times for cars waiting NS
        vect_car_EW = [x + wt for x in vect_car_EW] + [wt - x for x in Vec_EW[1]] 

    print("\tAverage waiting time EW: "+ str(EW_wt) +" \n\tAverage waiting time NS:" + str(NS_wt))
        
def trials(num_trials, smallest_wt, largest_wt):
    x = sorted(np.random.uniform(smallest_wt, largest_wt, num_trials))
    for i in x:
        i = int(i)
        print("For North South traffic light of length " + str(i) + "sec, we get:")
        simulate(i)        

trials(10, 45, 55)



